## Requirements
On importe toutes les librairies nécéssaires

In [5]:
import numpy as np
import re
import pandas as pd
#!pip install unidecode
from unidecode import unidecode
import torch
import ast

In [6]:
from googlesearch import search

In [7]:
from PyPDF2 import PdfReader

In [8]:
import os
pd.set_option('mode.chained_assignment', None)
import s3fs

In [9]:
import matplotlib.pyplot as plt

## Fonctions

In [10]:
def normalize_text(s):
    """removing caractères, /, accents, m3, /n"""
    import string, re
    
    def no_unit(text):
        regex="m3"
        return re.sub(regex, " ", text)
    def no_exp(text):
        regex="m³"
        return re.sub(regex, " ", text)
    def no_punct(text):
        regex="\\.(?!\d)"
        return re.sub(regex," ",text)
   
    def white_space_fix(text):
        return " ".join(text.split())
    def no_date(text):
        date_extract_pattern = "[0-9]{1,2}\\/[0-9]{1,2}\\/[0-9]{4}"
        return re.sub(date_extract_pattern,"",text)
    def special_char(text):
        regex=regex = "[^\w\s,.]"
        return re.sub(regex, " ", text)
   #on retire tout les caractère des réponses 
    def no_char(text):
        regex=r"([a-zA-Z])"
        return re.sub(regex, " ", text)
    def comma(text):
        regex=",(?!\d)"
        return re.sub(regex," ",text)
    def point(text):
        return re.sub("\,",".",text)
    def indic(text):
        regex="(P\d\d\d\\.\d\s*)|(D\d\d\d\\.\d\s*)|(VP\\.\d\d\d)"
        return re.sub(regex," ", text)

    return white_space_fix(point(no_punct(no_char(unidecode(comma(special_char(indic(no_exp(no_unit(no_date(s)))))))))))

In [11]:
def normalize(s):
    """removing caractères, /, accents, m3, /n"""
    import string, re
    def no_punct(text):
        regex="\\.(?!\d)"
        return re.sub(regex," ",text)
    def white_space_fix(text):
        return " ".join(text.split())
    def special_char(text):
        regex=regex = "[^\w\s,.]"
        return re.sub(regex, " ", text)
    def comma(text):
        regex=",(?!\d)"
        return re.sub(regex," ",text)
    def lower(text):
        return text.lower()

    return white_space_fix(no_punct(unidecode(lower(special_char(s)))))

In [12]:
def normalize_an(s):
    """removing caractères, /, accents, m3, /n"""
    import string, re
    
    def no_unit(text):
        regex="m3"
        return re.sub(regex, " ", text)
    def no_exp(text):
        regex="m³"
        return re.sub(regex, " ", text)
    def no_punct(text):
        regex="\\.(?!\d)"
        return re.sub(regex," ",text)
   
    def white_space_fix(text):
        return "".join(text.split())
    def no_date(text):
        date_extract_pattern = "[0-9]{1,2}\\/[0-9]{1,2}\\/[0-9]{4}"
        return re.sub(date_extract_pattern,"",text)
    def special_char(text):
        regex=regex = "[^\w\s,.]"
        return re.sub(regex, " ", text)
   #on retire tout les caractère des réponses 
    def no_char(text):
        regex=r"([a-zA-Z])"
        return re.sub(regex, " ", text)
    def comma(text):
        regex=",(?!\d)"
        return re.sub(regex," ",text)
    def point(text):
        return re.sub("\,",".",text)
    def indic(text):
        regex="(P\d\d\d\\.\d\s*)|(D\d\d\d\\.\d\s*)|(VP\\.\d\d\d)"
        return re.sub(regex," ", text)

    return white_space_fix(point(no_punct(no_char(unidecode(comma(special_char(indic(no_exp(no_unit(no_date(s)))))))))))

In [13]:
def white_space_fix(text):
        return " ".join(text.split())

In [14]:
def lower(text):
        return text.lower()

In [15]:
def saut(text):
        regex="\n"
        return re.sub(regex,"",text)

In [16]:
def compute_f1(prediction, truth):
    pred_tokens =normalize(prediction).split()
    truth_tokens = normalize(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    #print(len(common_tokens))
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

## Charger les modèles
On charge le modèle de question réponse d'etalab pour déceler le nom de la ville, l'année et la compétence mentionnée par le rapport

In [17]:
from transformers import pipeline
nlp = pipeline('question-answering', model='etalab-ia/camembert-base-squadFR-fquad-piaf', tokenizer='etalab-ia/camembert-base-squadFR-fquad-piaf')

## Pré-filtres

In [18]:
#  Fonction extrait l'intro
def extract_intro(pdf_path):
    response = requests.get(pdf_path)
    with io.BytesIO(response.content) as f:
        reader = PdfReader(f)
        page = reader.pages
        text = []
        for i in range(0, 5):
            page = reader.pages[i]
            output = page.extract_text()
            text.append(output)
            text1 = ''
        for x in text:
            text1 += ' ' + x
    return text1

In [67]:
def prefiltres(text1):
    global df
   
    
    # année
    année = nlp({'question': "Quelle est l'année d'exercice du rapport ?", 
    'context': text1})
    an = normalize_an(année['answer'])
    #print(an)

    # compétence
    reau= "eau potable"
    rass="assainissement"

    d=nlp({'question': "Quels types de services ? ",
         'context': text1})
    #print(d)
    prediction=d["answer"]
    prediction=normalize(prediction)
    if ((re.findall(reau,prediction)==[])&(re.findall(rass,prediction)==[])):
        d=nlp({'question': "eau potable ou assainissement collectif ? ",
         'context': text1})
        #print(d)
        prediction=d["answer"]
        prediction=normalize(prediction)
        if ((re.findall(reau,prediction)==[])&(re.findall(rass,prediction)==[])):
            d=nlp({'question': "Quelle est l'entité de gestion ? ",
             'context': text1})
            #print(d)
            prediction=d["answer"]
            prediction=normalize(prediction)
            if ((re.findall(reau,prediction)==[])&(re.findall(rass,prediction)==[])):
                print("type de service non retrouvé dans le texte")
    truth="""eau potable"""
    prediction=d['answer']
    #print(prediction)
    f1_ep=compute_f1(prediction,truth)
    #print('f1 eau potable:',f1_ep)

    truth="""assainissement"""
    prediction=normalize(d['answer'])
    f1_ac=compute_f1(prediction,truth)
    #print('f1 assainissement:',f1_ac)

    if ((re.findall(reau,prediction)!=[])&(re.findall(rass,prediction)!=[])):
        #print(re.findall(reau,prediction),re.findall(rass,prediction))
        
        service="AC+EP"
    else:
        if (f1_ep > f1_ac ):
            service="EP"
        if (f1_ep < f1_ac):
            service="AC"
        if(f1_ep==0)&(f1_ac==0):
             service="Not defined"
    print(service)
    
  
    #ville
    d_ville=nlp({'question': "Quel est le nom de la commune?",
         'context': text1})
    ville=d_ville['answer']
    ville=normalize(ville)
    print(ville)
    if (ville not in df_ville["Nom_de_la_commune"].values):
        d_ville=nlp({'question': "Quel est le nom de la ville?",
         'context': text1})
        ville=d_ville['answer']
        ville=normalize(ville)
        print(ville)
        if (ville not in df_ville["Nom_de_la_commune"].values):
          print("Ville not found")
    print(ville)

    return(service,an,ville)

## Charger le fichier d'URL

In [34]:
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "/mberthe/narval"
FILE_KEY_Q = "camemBERT/collect_2023.csv"
FILE_PATH_Q = BUCKET + "/" + FILE_KEY_Q

with fs.open(FILE_PATH_Q,  mode="r") as file_in:
    collect_2023=pd.read_csv(file_in, sep= ";")


In [35]:
collect_2023["URL"].values

array([], dtype=object)

## Charger ville

In [38]:
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "/mberthe/narval"
FILE_KEY_Q = "camemBERT/df_ville.csv"
FILE_PATH_Q = BUCKET + "/" + FILE_KEY_Q

with fs.open(FILE_PATH_Q,  mode="r") as file_in:
    df_ville=pd.read_csv(file_in, sep= ";")


In [39]:
for i in df_ville.index:
    df_ville["Nom_de_la_commune"][i]=lower(df_ville["Nom_de_la_commune"][i])

## main

In [72]:
from googlesearch import search
requête='RPQS'
links=[]

for url in  search(requête,start=200,pause=2):
    print("URL: ",url)
    try:
        read = requests.get(url)
 
# full html content
        html_content = read.content
 
# Parse the html content
        soup = BeautifulSoup(html_content, "html.parser")
        for link in soup.find_all('a'):
            current_link = link.get('href')
        #print(current_link)
            if current_link!=None:
                if current_link.endswith('pdf'):
                    print("PDF_link",current_link)
                    links.append(current_link)
    except Exception:
        pass
links

[]

In [69]:
import requests
from bs4 import BeautifulSoup
import io
from PyPDF2 import PdfFileReader

In [70]:
for i in range(0,len(links)):
    if links[i] not in collect_2023["URL"].values:
        try :
            print(i)
            text1=extract_intro(links[i])
            #print("intro extract")
            service,an,ville = prefiltres(text1)
            print("lien",i,"an:", an,"ville:",ville)
            dict_temp={"URL":links[i],"ville":ville,"annee":an,"competence":service}
            df_result=pd.Series(dict_temp)
            df_result=df_result.to_frame()
            df_result=df_result.transpose()
            collect_2023=pd.concat([collect_2023,df_result])
        except:
            pass

In [71]:
len(links)

0

In [73]:
collect_2023

,URL,ville,annee,competence
0,https://www.seapan.fr//_medias/fichiers/rpqs-e...,benejacq,2020,AC
0,https://www.seapan.fr//_medias/fichiers/rpqs-a...,coarraze,2021,AC
0,https://www.seapan.fr//_medias/fichiers/rpqs-s...,benejacq,2020,AC
0,https://www.seapan.fr//_medias/fichiers/pac-pl...,monplai sir 64800 ben ejacq,2022,AC
0,https://www.seapan.fr//_medias/fichiers/rpqs-a...,"maison de l eau et de l assai nissement,",2020,AC
...,...,...,...,...
0,https://perpignanmediterraneemetropole.fr/wp-c...,perpignan,2019,Not defined
0,https://perpignanmediterraneemetropole.fr/wp-c...,peyrestortes,201820192013,Not defined
0,https://perpignanmediterraneemetropole.fr/wp-c...,peyrestortes,20172018,Not defined
0,https://perpignanmediterraneemetropole.fr/wp-c...,perpignan,2008,Not defined


In [74]:
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET_OUT = "mberthe/narval"
FILE_KEY_OUT_S3 = "camemBERT/collect_2023.csv"#modif nom csv
FILE_PATH_OUT_S3 = BUCKET_OUT + "/" + FILE_KEY_OUT_S3

with fs.open(FILE_PATH_OUT_S3, 'w') as file_out:
    collect_2023.to_csv(file_out,index=False,sep=';')